In [ ]:
!pip install optuna

     |████████████████████████████████| 256kB 2.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 8.7MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 112kB 11.3MB/s 
     |████████████████████████████████| 143kB 10.5MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for optuna: filename=optuna-2.2.0-cp36-none-any.whl size=340928 sha256=c4b506517820859c394a2289b06f9fc8ea1603d87da8634cab97be834465cd2c
  Stored in directory: /root/.cache/pip/wheels/a6/27/26/3ef57a8fb2c8bceef3f4ece73483cec1895e06eae442d34a41
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.0-cp36-none-any.whl size=8693 sha256=716f45f475ec893be1a45b7751563497249a10cb61f9c6db970ddcf2355fae93
  Stored in directory: /root/.cache/pip/wheels/b2

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense 
from tensorflow.keras.optimizers import Adam
import optuna 

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)


(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
x_train = x_train/255
x_test = x_test/255


In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10) 

In [ ]:
ModelMLP = Sequential (
    [
     Flatten(input_shape=[28,28]),
     Dense(128, activation = 'relu'),
     Dense(64, activation = 'relu'),
     Dense(10, activation = 'softmax')
     
    ]
    )

In [ ]:
ModelMLP.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ModelMLP.compile (keras.optimizers.Adam(learning_rate=1e-3 ), loss ='categorical_crossentropy', metrics= ['accuracy'])


In [ ]:
study = optuna.create_study(storage='sqlite:///model_selection.db', study_name='model_selection', direction='maximize', load_if_exists=True)

[I 2020-10-08 11:01:50,877] A new study created in RDB with name: model_selection


In [ ]:
class MyException(Exception):
  pass
def getModel (hidden_layers_n, n_hidden):

  ModelMLP = Sequential (
      [
      Flatten(input_shape=[28,28]), 
      ]
      )
  
  for i in range (n_hidden):
    ModelMLP.add (Dense(hidden_layers_n, activation = 'relu')),
  ModelMLP.add(Dense(10, activation = 'softmax'))
  return ModelMLP
 

def objective(trial):
  hidden_layers_n = trial.suggest_int('hidden_layers_n', low=60, high=150)
  n_hidden = trial.suggest_int('n_hidden', low=1, high=5)
  model = getModel(hidden_layers_n, n_hidden)
  # creazione della rete con gli iperparametri scelti
  # fit
  model.compile (keras.optimizers.Adam(learning_rate=1e-3 ), loss ='categorical_crossentropy', metrics= ['accuracy'])
  model.fit(x_train, y_train, epochs= 10, validation_split=0.2, batch_size=32)
  # metrica che mi interessa sul validation set
 
  return model.history.history['val_accuracy'][-1]
  
   

In [ ]:
study.optimize(objective, n_trials = 100)

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5600 - accuracy: 0.7962 - val_loss: 0.4340 - val_accuracy: 0.8425
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4035 - accuracy: 0.8524 - val_loss: 0.3886 - val_accuracy: 0.8580
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3624 - accuracy: 0.8675 - val_loss: 0.3611 - val_accuracy: 0.8695
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3423 - accuracy: 0.8757 - val_loss: 0.3849 - val_accuracy: 0.8661
Epoch 5/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3206 - accuracy: 0.8816 - val_loss: 0.3483 - val_accuracy: 0.8708
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3071 - accuracy: 0.8857 - val_loss: 0.3499 - val_accuracy: 0.8738
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2984 - accuracy: 0.8884 - val_loss: 0.3488 - val_accuracy:

[I 2020-10-08 11:04:03,054] Trial 1 finished with value: 0.8812500238418579 and parameters: {'hidden_layers_n': 68, 'n_hidden': 5}. Best is trial 1 with value: 0.8812500238418579.


Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5120 - accuracy: 0.8145 - val_loss: 0.4234 - val_accuracy: 0.8443
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3753 - accuracy: 0.8629 - val_loss: 0.3838 - val_accuracy: 0.8598
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3413 - accuracy: 0.8724 - val_loss: 0.3694 - val_accuracy: 0.8662
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3176 - accuracy: 0.8815 - val_loss: 0.3459 - val_accuracy: 0.8698
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2975 - accuracy: 0.8888 - val_loss: 0.3413 - val_accuracy: 0.8737
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2837 - accuracy: 0.8930 - val_loss: 0.3334 - val_accuracy: 0.8811
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2710 - accuracy: 0.8976 - val_loss: 0.3199 - val_accuracy:

[I 2020-10-08 11:04:59,643] Trial 2 finished with value: 0.8838333487510681 and parameters: {'hidden_layers_n': 137, 'n_hidden': 3}. Best is trial 2 with value: 0.8838333487510681.


Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5260 - accuracy: 0.8158 - val_loss: 0.4093 - val_accuracy: 0.8534
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3929 - accuracy: 0.8582 - val_loss: 0.3754 - val_accuracy: 0.8633
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3545 - accuracy: 0.8721 - val_loss: 0.3788 - val_accuracy: 0.8647
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3294 - accuracy: 0.8803 - val_loss: 0.3495 - val_accuracy: 0.8733
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3106 - accuracy: 0.8875 - val_loss: 0.3528 - val_accuracy: 0.8739
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2954 - accuracy: 0.8916 - val_loss: 0.3532 - val_accuracy: 0.8749
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2827 - accuracy: 0.8951 - val_loss: 0.3356 - val_accuracy:

[I 2020-10-08 11:05:51,006] Trial 3 finished with value: 0.8820000290870667 and parameters: {'hidden_layers_n': 86, 'n_hidden': 1}. Best is trial 2 with value: 0.8838333487510681.


Epoch 1/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.5069 - accuracy: 0.8211 - val_loss: 0.3937 - val_accuracy: 0.8580
Epoch 2/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3790 - accuracy: 0.8614 - val_loss: 0.3823 - val_accuracy: 0.8587
Epoch 3/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3414 - accuracy: 0.8758 - val_loss: 0.3422 - val_accuracy: 0.8770
Epoch 4/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3155 - accuracy: 0.8819 - val_loss: 0.3650 - val_accuracy: 0.8651
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2978 - accuracy: 0.8889 - val_loss: 0.3285 - val_accuracy: 0.8852
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2834 - accuracy: 0.8936 - val_loss: 0.3280 - val_accuracy: 0.8824
Epoch 7/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2695 - accuracy: 0.8992 - val_loss: 0.3221 - val_accuracy:

[I 2020-10-08 11:06:44,595] Trial 4 finished with value: 0.8900833129882812 and parameters: {'hidden_layers_n': 107, 'n_hidden': 2}. Best is trial 4 with value: 0.8900833129882812.


Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5355 - accuracy: 0.8081 - val_loss: 0.4247 - val_accuracy: 0.8456
Epoch 2/10
  94/1500 [>.............................] - ETA: 4s - loss: 0.4278 - accuracy: 0.8454

KeyboardInterrupt: ignored

In [ ]:
ModelMLP.evaluate(x_test, y_test)